In [1]:
import pandas as pd
import xlwings as xw
from xlwings.constants import AutoFillType

In [2]:
import os
from databricks.connect import DatabricksSession
import pyspark.sql.functions as F
import pandas as pd

server_hostname = os.getenv("DATABRICKS_SERVER_HOSTNAME", "https://dbc-456d715c-6cb0.cloud.databricks.com")
http_path = os.getenv("DATABRICKS_HTTP_PATH", "/sql/1.0/warehouses/3bdbcae0f85f2f83")
access_token = os.getenv("DATABRICKS_TOKEN", "<your-access-token>")
cluster_id = "0919-213318-ofm4tbtn"

    spark = DatabricksSession.builder.remote(
        host=server_hostname,
        token=access_token,
        cluster_id=cluster_id
    ).getOrCreate()
    
    try:
        income_statement_df = spark.read.table("bronze.ns_netsuite.netsuite2__income_statement")

    # Define the start and end periods (replace with your actual values)
    start_period = "2024-01-01"  # Replace with your {{start_period}}
    end_period = "2024-12-31"    # Replace with your {{end_period}}

    # Define the sort_order mapping
    sort_order_mapping = {
        "4105": 10, "4106": 20, "4107": 30, "4110": 40, "4110A": 50, "4110B": 60, "4110C": 70, "4111": 75,
        "4120": 80, "4120A": 90, "4120B": 100, "4120C": 110, "4121": 120, "4130": 130, "4130C": 140,
        "4141": 150, "4142": 155, "4190": 160, "5110": 170, "5110A": 180, "5120A": 190, "5120": 200,
        "5301": 210, "5305": 220, "5304": 230, "5320": 240, "5330": 250, "5340": 260, "5400": 270,
        "5520": 280, "5401": 290, "5402": 300, "5141": 310, "5530": 320, "5400": 325, "5403": 330,
        "5404": 340, "5199": 350, "5510": 360, "6000": 370, "6010": 380, "6011": 390, "6013": 400,
        "6030": 410, "6040": 420, "6041": 430, "6100": 440, "6101": 450, "6110": 460, "6200": 470,
        "6510": 480, "6520": 490, "6530": 500, "6540": 510, "6550": 520, "6560": 530, "6570": 540,
        "7005": 550, "7006": 560, "7100": 570, "7110": 580, "7120": 590, "7125": 600, "7130": 610,
        "7131": 620, "7140": 630, "7141": 640, "7150": 650, "7160": 660, "7170": 670, "7171": 680,
        "7172": 690, "7174": 700, "7175": 710, "7176": 720, "7180": 730, "7190": 740, "8000": 750,
        "9001": 760, "9001D": 770, "8010": 780, "8011": 790, "9007": 800, "9010": 810
    }

    # Apply the transformations
    result = (income_statement_df
        # Filter conditions
        .filter((F.col("accounting_period_ending") > start_period) &
                (F.col("accounting_period_ending") <= end_period) &
                (~F.col("account_number").isin(["4100", "4105V", "4199", "4200", "5101", "5111", 
                                                "5121", "5210", "4142", "5100", "5110", "5110A", 
                                                "5120A", "9000", "9005"])))
        # Select and transform columns
        .groupBy(
            "accounting_period_ending",
            "account_number",
            "account_number_and_name"
        )
        .agg(
            F.sum("transaction_amount").alias("amount")
        )
        .withColumn(
            "account_number_and_name",
            F.concat(F.lit(""), F.col("account_number_and_name"))
        )
        .withColumn(
            "sort_order",
            F.when(F.col("account_number").isin(list(sort_order_mapping.keys())),
                   F.lit(None))  # Placeholder for the mapping
        )
    )

    # Apply the sort_order mapping using multiple when() clauses
    for account, order in sort_order_mapping.items():
        result = result.withColumn(
            "sort_order",
            F.when(F.col("account_number") == account, order).otherwise(F.col("sort_order"))
        )
    result = result.withColumn(
        "sort_order",
        F.when(F.col("sort_order").isNull(), 900).otherwise(F.col("sort_order"))
    )

    # Order the results
    income_statement_df = result.orderBy("accounting_period_ending", "sort_order")
    
except Exception as e:
    print(f"Error executing query: {e}")
# finally:
#     # Databricks Connect automatically manages the session, but you can stop it if needed
#     spark.stop()

In [3]:
# Function to indent specific account labels
def indent_account_labels(df_pivot, accounts, indent_level):
    """
    Applies indentation to specific account labels without affecting other functionality.
    
    :param df_pivot: The dataframe containing the accounts
    :param accounts: List of account numbers to indent
    :param indent_level: The level of indentation to apply (each level is 2 non-breaking spaces)
    :return: The dataframe with indented labels
    """
    # Create the indentation prefix
    prefix = '\u00A0' * (2 * indent_level)
    
    # Make a copy to avoid modifying during iteration issues
    rows_to_modify = []
    for i, row in df_pivot.iterrows():
        if row['account_number'] in accounts:
            rows_to_modify.append(i)
    
    # Apply indentation to identified rows
    for i in rows_to_modify:
        current_label = df_pivot.at[i, 'account_number_and_name']
        # Remove any existing indentation
        while current_label.startswith('\u00A0'):
            current_label = current_label[1:]
        # Apply new indentation
        df_pivot.at[i, 'account_number_and_name'] = prefix + current_label
    
    return df_pivot

def insert_summary_rows(df_pivot, df, accounts, label, insert_at_sort_order=None, indents=0, indent_children=False):
    """
    Inserts a summary row below accounts that belong to the given list.

    :param df_pivot: The pivoted dataframe.
    :param df: The original dataframe.
    :param accounts: A list of account numbers to be summed.
    :param label: The label for the summary row.
    :param insert_at_sort_order: The sort_order value to insert the row at (will insert before the first row with sort_order >= this value)
    :param indents: Number of indentations to apply (each indent is 2 spaces)
    :param indent_children: Whether to indent the child rows one level deeper than the summary row
    :return: The updated dataframe with the summary row.
    """
    
    # Filter the dataframe for the given accounts
    filtered_df = df[df["account_number"].isin(accounts)]
    if filtered_df.empty:
        return df_pivot  # Skip if no accounts match

    # Apply indentation to the label
    indented_label = '\u00A0' * (2 * indents) + label

    # Sum the amounts by accounting_period_ending
    summary_values = filtered_df.groupby("accounting_period_ending")["amount"].sum().reset_index()
    
    # Create a new row
    summary_row = pd.DataFrame({
        "sort_order": insert_at_sort_order,  # Set the sort_order to the insertion point
        "account_number": "",
        "account_number_and_name": indented_label,
        "is_original": False
    }, index=[0])
    
    # Add the summary values for each period
    for _, row in summary_values.iterrows():
        period = row["accounting_period_ending"]
        summary_row[period] = row["amount"]
    
    # Apply child indentation if requested
    if indent_children:
        # Calculate child indentation level (one level deeper)
        child_indent_level = indents + 1
        df_pivot = indent_account_labels(df_pivot, accounts, child_indent_level)
    
    # Now insert the summary row based on sort_order
    if insert_at_sort_order is not None:
        # Find the first row with sort_order >= insert_at_sort_order
        for i, row in df_pivot.iterrows():
            if pd.notnull(row['sort_order']) and row['sort_order'] >= insert_at_sort_order:
                # Insert before this row
                df_pivot = pd.concat([df_pivot.iloc[:i], summary_row, df_pivot.iloc[i:]], ignore_index=True)
                return df_pivot
        
        # If we didn't find a suitable position, append at the end
        df_pivot = pd.concat([df_pivot, summary_row], ignore_index=True)
    else:
        # If no sort_order specified, append at the end
        df_pivot = pd.concat([df_pivot, summary_row], ignore_index=True)

    return df_pivot


# Function to add a header/title row with no numeric values
def insert_header_row(df_pivot, label, insert_at_sort_order=None, indents=0):
    """
    Inserts a header/title row with just a label and no numeric values.
    
    :param df_pivot: The pivoted dataframe.
    :param label: The label for the header row.
    :param insert_at_sort_order: The sort_order value to insert the row at (will insert before the first row with sort_order >= this value)
    :param indents: Number of indentations to apply (each indent is 2 spaces)
    :return: The updated dataframe with the header row.
    """

    # Apply indentation to the label
    indented_label = '\u00A0' * (2 * indents) + label

    # Create a new row with the label
    header_row = pd.DataFrame({
        "sort_order": insert_at_sort_order,  # Set the sort_order to the insertion point
        "account_number": "",
        "account_number_and_name": indented_label,
        "is_original": False  # Mark as not an original row
    }, index=[0])
    
    # Get all numeric columns
    numeric_columns = [col for col in df_pivot.columns 
                     if col not in ["sort_order", "account_number", "account_number_and_name", "is_original"]]
    
    # Set empty string for all numeric columns
    for col in numeric_columns:
        header_row[col] = ""
    
    # Determine insertion position based on sort_order
    if insert_at_sort_order is not None:
        # Find the first row with sort_order >= insert_at_sort_order
        for i, row in df_pivot.iterrows():
            if pd.notnull(row['sort_order']) and row['sort_order'] >= insert_at_sort_order:
                # Insert before this row
                df_pivot = pd.concat([df_pivot.iloc[:i], header_row, df_pivot.iloc[i:]], ignore_index=True)
                return df_pivot
        
        # If we didn't find a suitable position, append at the end
        df_pivot = pd.concat([df_pivot, header_row], ignore_index=True)
    else:
        # If no sort_order specified, append at the end
        df_pivot = pd.concat([df_pivot, header_row], ignore_index=True)
        
    return df_pivot

# Format numbers as currency
def format_currency(value):
    if pd.isna(value) or value == "":
        return ""
    try:
        float_value = float(value)
        return f"(${abs(float_value):,.2f})" if float_value < 0 else f"${float_value:,.2f}"
    except (ValueError, TypeError):
        return value  # Return original value if it can't be converted to float


# Format numbers 
def format_number(value):
    if pd.isna(value) or value == "":
        return ""
    try:
        float_value = float(value)
        return f"({abs(float_value):,.2f})" if float_value < 0 else f"{float_value:,.2f}"
    except (ValueError, TypeError):
        return value  # Return original value if it can't be converted to float




In [4]:
# Assume df is the SQL query result
# df_pivot = income_statement_df.pivot(index=["sort_order", "account_number", "account_number_and_name"], columns="accounting_period_ending", values="amount").fillna(0)
# Instead of using pivot which creates a multi-index, use pivot_table and then reset_index
# income_statement_df = income_statement_df.limit(1000).toPandas()
income_statement_df = income_statement_df.toPandas()
df_pivot = income_statement_df.pivot_table(
    index=["sort_order", "account_number", "account_number_and_name"],
    columns="accounting_period_ending",
    values="amount",
    aggfunc='sum',  # Explicitly defining the aggregation function
    fill_value=0
).reset_index()  # This converts the index back to columns

df_pivot['is_original'] = True

# Manually define account groupings
account_summaries = {
    "TOTAL - REV Reserve": {
        "accounts": ["4105", "4106", "4107"],
        "insert_at_sort_order": 35,
        "indents": 3,
        "indent_children": True
    },
    "TOTAL - REV VSC": {
        "accounts": ["4110", "4110A", "4110B", "4110C", "4111"],
        "insert_at_sort_order": 77,
        "indents": 3,
        "indent_children": True
    },
    "TOTAL - REV GAP": {
        "accounts": ["4120","4120A","4120B","4120C","4121"],
        "insert_at_sort_order": 125,
        "indents": 3,
        "indent_children": True
    },
    "TOTAL - REV DOC Fees": {
        "accounts": ["4130","4130C"],
        "insert_at_sort_order": 145,
        "indents": 3,
        "indent_children": True
    },
    "TOTAL - REV - LEASE BUYOUT": {
        "accounts": ["4105", "4106", "4107","4110", "4110A", "4110B", "4110C", "4111","4120","4120A","4120B","4120C","4121","4130","4130C","4141","4142","4190"],
        "insert_at_sort_order": 166,
        "indents": 1,
        "indent_children": False
    },
    "TOTAL - COR - Other": {
        "accounts": ["5110","5110A","5120A","5120"],
        "insert_at_sort_order": 206,
        "indents": 3,
        "indent_children": True
    },
    "TOTAL - COR - Direct People Cost": {
        "accounts": ["5301","5304","5305","5320","5330","5340"],
        "insert_at_sort_order": 266,
        "indents": 3,
        "indent_children": True
    },
    "TOTAL - COR - Payoff Expense": {
        "accounts": ["5400","5520"],
        "insert_at_sort_order": 286,
        "indents": 3,
        "indent_children": True
    },
    "TOTAL - COR - Registration Expense": {
        "accounts": ["5402","5141","5530"],
        "insert_at_sort_order": 325,
        "indents": 3,
        "indent_children": True
    },
    "TOTAL - PAYOFF Expense": {
        "accounts": ["5400","5520","5401","5402","5141","5530","5403","5404"],
        "insert_at_sort_order": 344,
        "indents": 2,
        "indent_children": False
    },
    "TOTAL - COR - Other": {
        "accounts": ["5199", "5510"],
        "insert_at_sort_order": 364,
        "indents": 3,
        "indent_children": True
    },
    "TOTAL - COST OF REVENUE": {
        "accounts": ["5110","5110A","5120A","5120","5301","5304","5305","5320","5330","5340","5400","5401","5520","5402","5141","5530","5403","5404","5199", "5510"],
        "insert_at_sort_order": 365,
        "indents": 2,
        "indent_children": False
    },
    "GROSS PROFIT": {
        "accounts": ["4105", "4106", "4107","4110", "4110A", "4110B", "4110C", "4111","4120","4120A","4120B","4120C","4121","4130","4130C","4141","4190","5110","5110A","5120A","5120","5301","5304","5305","5320","5330","5340","5400","5401","5520","5402","5141","5530","5403","5404","5199", "5510"],
        "insert_at_sort_order": 366,
        "indents": 0,
        "indent_children": False
    },
    
    "TOTAL - PEOPLE COST": {
        "accounts": ["6000", "6010", "6011", "6013", "6030", "6040", "6041", "6100", "6101", "6110", "6200"],
        "insert_at_sort_order": 475,
        "indents": 3,
        "indent_children": True
    },
    "TOTAL - MARKETING": {
        "accounts": ["6510","6520","6530","6540","6550","6560","6570"],
        "insert_at_sort_order": 545,
        "indents": 3,
        "indent_children": True
    },
    "TOTAL - G&A EXPENSE": {
        "accounts": ["7005", "7006", "7100", "7110", "7120", "7125", "7130", "7131", "7140", "7141", "7150", "7160", "7170",
                     "7171", "7172", "7174", "7175", "7176", "7180", "7190", "8000"],
        "insert_at_sort_order": 755,
        "indents": 3,
        "indent_children": True
    },
    "TOTAL EXPENSE": {
        "accounts": ["6000", "6010", "6011", "6013", "6030", "6040", "6041", "6100", "6101", "6110", "6200","6510","6520","6530","6540","6550","6560","6570","7005", "7006", "7100", "7110", "7120", "7125", "7130", "7131", "7140", "7141", "7150", "7160", "7170",
                     "7171", "7172", "7174", "7175", "7176", "7180", "7190", "8000"],
        "insert_at_sort_order": 756,
        "indents": 1,
        "indent_children": False
    },
    "Net Ordinary Revenue": {
        "accounts": ["4105", "4106", "4107","4110", "4110A", "4110B", "4110C", "4111","4120","4120A","4120B","4120C","4121","4130","4130C","4141","4190","5110","5110A","5120A","5120","5301","5304","5305","5320","5330","5340","5400","5401","5520","5402","5141","5530","5403","5404","5199","5510","6000", "6010", "6011", "6013", "6030", "6040", "6041", "6100", "6101", "6110", "6200","6510","6520","6530","6540","6550","6560","6570","7005", "7006", "7100", "7110", "7120", "7125", "7130", "7131", "7140", "7141", "7150", "7160", "7170",
                     "7171", "7172", "7174", "7175", "7176", "7180", "7190", "8000"],
        "insert_at_sort_order": 757,
        "indents": 0,
        "indent_children": False
    },
    "TOTAL - Other Income": {
        "accounts": ["9001", "9001D"],
        "insert_at_sort_order": 775,
        "indents": 3,
        "indent_children": True
    },
    "TOTAL - Other Expense": {
        "accounts": ["8010", "8011", "9007", "9010"],
        "insert_at_sort_order": 814,
        "indents": 3,
        "indent_children": True
    },
    "TOTAL - Net Other Income": {
        "accounts": ["9001", "9001D", "8010", "8011", "9007", "9010"],
        "insert_at_sort_order": 820,
        "indents": 1,
        "indent_children": False
    },
    "NET INCOME": {
        "accounts": ['4105','4106','4107','4110','4110A','4110B','4110C','4111','4120','4120A','4120B','4120C','4121','4130','4130C','4141',"4142",'4190','5110','5110A','5120A','5120','5301','5305','5304','5320','5330','5340','5400','5520','5401','5402','5141','5530','5400','5403','5404','5199','5510','6000','6010','6011','6013','6030','6040','6041','6100','6101','6110','6200','6510','6520','6530','6540','6550','6560','6570','7005','7006','7100','7110','7120','7125','7130','7131','7140','7141','7150','7160','7170','7171','7172','7174','7175','7176','7180','7190','8000','9001','9001D','8010','8011','9007','9010'],
        "insert_at_sort_order": 830,
        "indents": 0,
        "indent_children": False
    },
}

df_pivot = indent_account_labels(df_pivot, ["4141", "4190","5401","5403","5404"], indent_level=3)

df_pivot = insert_header_row(df_pivot, "Ordinary Revenue/Expense", insert_at_sort_order=5, indents=0)
df_pivot = insert_header_row(df_pivot, "REVENUE", insert_at_sort_order=6, indents=1)
df_pivot = insert_header_row(df_pivot, "BUYOUT", insert_at_sort_order=7, indents=2)
df_pivot = insert_header_row(df_pivot, "PAYOFF EXPENSE", insert_at_sort_order=267, indents=2)
df_pivot = insert_header_row(df_pivot, "EXPENSE", insert_at_sort_order=368, indents=0)
df_pivot = insert_header_row(df_pivot, "PEOPLE COST", insert_at_sort_order=369, indents=2)
df_pivot = insert_header_row(df_pivot, "MARKETING", insert_at_sort_order=476, indents=2)
df_pivot = insert_header_row(df_pivot, "G&A EXPENSE", insert_at_sort_order=546, indents=2)
df_pivot = insert_header_row(df_pivot, "Other Income & Expense", insert_at_sort_order=758, indents=0)

for label, data in account_summaries.items():
    indent_children = data.get("indent_children", False)  # Default to False if not specified
    df_pivot = insert_summary_rows(
        df_pivot, 
        income_statement_df, 
        data["accounts"], 
        label, 
        insert_at_sort_order=data["insert_at_sort_order"], 
        indents=data["indents"],
        indent_children=indent_children
    )
    
df_pivot

# Format all numeric columns with currency formatting
numeric_columns = [col for col in df_pivot.columns if col not in ["sort_order", "account_number", "account_number_and_name", "is_original"]]

# Calculate total column BEFORE formatting the numbers
df_pivot['Total'] = df_pivot[numeric_columns].sum(axis=1)


# Apply formatting to all numeric columns including the Total column
for col in numeric_columns + ['Total']:
    if col in df_pivot.columns:
        df_pivot[col] = df_pivot.apply(
            lambda row: format_currency(row[col]) if row['is_original'] or row[col] != "" else "", 
            axis=1
        )

# if not _show_details:
#     # Filter to only show summary rows (non-original rows)
#     df_pivot = df_pivot[~df_pivot['is_original']]

# Clean up columns for display
# df_pivot = df_pivot.drop(columns=["is_original", "sort_order", "account_number"])

# Ensure all column names are strings for consistency
df_pivot.columns = [str(col) for col in df_pivot.columns]

# This should display directly in Hex
# df_pivot.show()



    

In [5]:
# Path to your existing Excel file (adjust the path as needed)
file_path = "/Users/david/financials.xlsx"

wb = xw.Book(file_path)

# wb = xw.Book()
#creates a worksheet object assigns it to ws
ws = wb.sheets["Income Statement Data"]

# pandas_df = income_statement_df.limit(1000).toPandas() 
ws.range("A1").value = df_pivot

ws.autofit()

ws.range("D:D").font.bold = True

last_row = ws.range("A1,A1").end('down').row


2